Implmenting section 4 "Hiding the Internal State of the Algorithm" in:

Golle P. (2006) A Private Stable Matching Algorithm. In: Di Crescenzo G., Rubin A. (eds) Financial Cryptography and Data Security. FC 2006. Lecture Notes in Computer Science, vol 4107. Springer, Berlin, Heidelberg. https://doi-org.proxygw.wrlc.org/10.1007/11889663_5

Let $A_1...A_3$ denote 3 men and $B_1...B_3$ denote three women.

Preferences:

$A_1$: $a_1=(0,2,1)$

$A_2$: $a_2=(1,2,0)$

$A_3$: $a_3=(2,1,0)$

---

$B_1$: $b_1=(1,2,0)$

$B_2$: $b_2=(0,2,1)$

$B_3$: $b_3=(0,1,2)$

In [89]:
import random
import sys

In [90]:
n = 3

In [177]:
a_1 = [0, 2, 1]
a_2 = [1, 2, 0]
a_3 = [2, 1, 0] 
men_preferences = {"a_1" : a_1, "a_2" : a_2, "a_3" : a_3}

b_1 = [1, 2, 0]
b_2 = [0, 2, 1]
b_3 = [0, 1, 2]
women_preferences = {"b_1" : b_1, "b_2" : b_2, "b_3" : b_3}

In [178]:
def hidden_internal_state(men_prefs, women_prefs):
    # Adding an additional n "fake" men
    for i in range(n):
        fake_man = [0, 1, 2] # The "fake" men prefs are unimportant
        men_prefs["a_{}".format((i + 1) + n)] = fake_man
        
        # Augment women preferences
        for woman in women_prefs:
            women_prefs[woman].append(i + n)
    
    # Computing a stable match
    e_k = [] # Set of engaged men
    f_k = [] # Set of free men
    engagements = {} # Engagement tracker
    
    e_k = [man for man in men_preferences][n:] # Initially, all fake men are engaged
    # We arbitrarily assign these fake men partners
    i = 0
    for fake_man in e_k:
        engagements[fake_man] = list(women_preferences.keys())[i]
        i = i + 1
    
    f_k = [man for man in men_preferences][:n] # Initially, all real men are free
    
    proposals_made = {} # Used to track proposals made by a man
    
    j = 0
    
    while f_k: # while f_k is not empty
        randomly_selected_man = random.choice(f_k)
        #randomly_selected_man = "a_3"
        his_prefs = men_prefs[randomly_selected_man]
        
        #his_first_choice = his_prefs.index(0) + 1
        his_prefs_sorted = his_prefs.copy()
        his_prefs_sorted.sort()
        his_first_choice = his_prefs.index(his_prefs_sorted[0]) + 1
        
        if his_prefs_sorted[0] == sys.maxsize * 2 + 1:
            f_k.remove(randomly_selected_man)
            
            print("{} OVERCOOKED. Free men: {}".format(randomly_selected_man, f_k))
            print("")
            print("---")
            print("")
            
            continue
        
        his_first_choice_name = "b_{}".format(his_first_choice)
        
        # Determine if he has proposed to her before
        if randomly_selected_man in proposals_made:
            print("YO")
        
        her_rank_of_him = women_prefs[his_first_choice_name][int(randomly_selected_man[-1:]) - 1]
        # https://stackoverflow.com/a/13149770
        her_current_partner = list(engagements.keys())[list(engagements.values()).index(his_first_choice_name)]
        her_rank_of_current_partner = women_prefs[his_first_choice_name][int(her_current_partner[-1:]) - 1]
        
        print("Randomly selected man: ", randomly_selected_man)
        print("His prefs: ", his_prefs)
        print("His first choice: ", his_first_choice)
        print("His first choice name: ", his_first_choice_name)
        print("Her rank of him: ", her_rank_of_him)
        print("Current engagements: ", engagements)
        print("Her current partner: ", her_current_partner)
        print("her prefs: ", women_prefs)
        print("her rank of current partner: ", her_rank_of_current_partner)
        
        if her_rank_of_him < her_rank_of_current_partner:
            engagements[randomly_selected_man] = his_first_choice_name
            del engagements[her_current_partner]
            e_k = list(engagements.keys())
            f_k[f_k.index(randomly_selected_man)] = her_current_partner
            his_prefs[his_first_choice - 1] = sys.maxsize * 2 + 1
            print("His prefs (post engagement): ", his_prefs)
        else:
            his_prefs[his_first_choice - 1] = sys.maxsize * 2 + 1
            print("His prefs (post (rejected) engagement): ", his_prefs)
        
        print("Current engagements: ", engagements)
        print("Set of engaged men: ", e_k)
        print("Set of free men: ", f_k)
        
        print("")
        print("---")
        print("")
        
        j = j + 1
        
        if(j == 100):
            break
   
    
    print(j)

In [179]:
hidden_internal_state(men_preferences, women_preferences)

Randomly selected man:  a_2
His prefs:  [1, 2, 0]
His first choice:  3
His first choice name:  b_3
Her rank of him:  1
Current engagements:  {'a_4': 'b_1', 'a_5': 'b_2', 'a_6': 'b_3'}
Her current partner:  a_6
her prefs:  {'b_1': [1, 2, 0, 3, 4, 5], 'b_2': [0, 2, 1, 3, 4, 5], 'b_3': [0, 1, 2, 3, 4, 5]}
her rank of current partner:  5
His prefs (post engagement):  [1, 2, 18446744073709551615]
Current engagements:  {'a_4': 'b_1', 'a_5': 'b_2', 'a_2': 'b_3'}
Set of engaged men:  ['a_4', 'a_5', 'a_2']
Set of free men:  ['a_1', 'a_6', 'a_3']

---

Randomly selected man:  a_6
His prefs:  [0, 1, 2]
His first choice:  1
His first choice name:  b_1
Her rank of him:  5
Current engagements:  {'a_4': 'b_1', 'a_5': 'b_2', 'a_2': 'b_3'}
Her current partner:  a_4
her prefs:  {'b_1': [1, 2, 0, 3, 4, 5], 'b_2': [0, 2, 1, 3, 4, 5], 'b_3': [0, 1, 2, 3, 4, 5]}
her rank of current partner:  3
His prefs (post (rejected) engagement):  [18446744073709551615, 1, 2]
Current engagements:  {'a_4': 'b_1', 'a_5': '